In [13]:
import mysql.connector
import csv
import pandas as pd
import os

In [14]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)

In [15]:
con = mysql.connector.connect(user='andr',
                              password='rstq!2Ro',
                              host='127.0.0.1',
                              database='cat_db',
                              auth_plugin='mysql_native_password'
                             )

In [16]:
curA = con.cursor(dictionary=True, buffered=True)
curB = con.cursor(dictionary=True, buffered=True)
curC = con.cursor(dictionary=True, buffered=True)

## First experiments
curA and curB shows the first few experiments on morphology, where we were not keeping track of the POS.

In [5]:
curA.execute("SELECT id_unigram, unigram, freq_all, morph FROM unigrams;")
rows = curA.fetchall()

In [6]:
with open('morph_tags.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['id_unigram', 'unigram', 'freq_all', 'morph'])
    for dictionary in rows:
        writer.writerow([dictionary['id_unigram'], dictionary['unigram'], dictionary['freq_all'], dictionary['morph']])

df = pd.read_csv('morph_tags.csv')

df.head()

,id_unigram,unigram,freq_all,morph
0,36215,NUM,94888,_
1,47683,<URL>,1307,_
2,381939,А,2,Animacy=Anim|Case=Nom|Gender=Fem|Number=Sing
3,381940,В,803,Animacy=Anim|Case=Nom|Gender=Fem|Number=Sing
4,381941,Малько,45,Animacy=Anim|Case=Nom|Gender=Fem|Number=Sing


In [7]:
curB.execute("SELECT morph, freq_all*count AS 'par_count' FROM (SELECT morph, freq_all, COUNT(*) AS 'count' FROM unigrams GROUP BY morph, freq_all) AS tab1")

In [8]:
rowsB = curB.fetchall()

In [9]:
with open('stats.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['morph', 'count'])
    for dictionary in rowsB:
        writer.writerow([dictionary['morph'], dictionary['par_count']])

In [10]:
df_par_stats = pd.read_csv('stats.csv')

df_par_stats[:10]

,morph,count
0,_,94888
1,_,1307
2,Animacy=Anim|Case=Nom|Gender=Fem|Number=Sing,536
3,Animacy=Anim|Case=Nom|Gender=Fem|Number=Sing,803
4,Animacy=Anim|Case=Nom|Gender=Fem|Number=Sing,90
5,_,175057
6,Animacy=Anim|Case=Nom|Gender=Fem|Number=Sing,107
7,Animacy=Anim|Case=Nom|Gender=Fem|Number=Sing,70
8,_,137292
9,Case=Nom|Degree=Pos|Number=Plur,49


## MySQL --> POS/tagset counts

With this MySQL search we extract from the database the frequency of use of each udpipe morphological tagset, preserving information about POS for each tagset. The aim is keeping a distinction between the different tagsets also based on the part of speech, because we assume POS behavior as disinct one from the other. 

In [17]:
curC.execute("""SELECT morph, pos, freq_all FROM
(SELECT morph, lemma, freq_all FROM unigrams) AS a JOIN
(SELECT id_lemmas, id_pos FROM lemmas) AS b ON lemma = id_lemmas JOIN pos ON b.id_pos = pos.id_pos;""")

In [18]:
rowsC = curC.fetchall()

In [19]:
with open('statistica.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['morph', 'POS', 'count'])
    for dictionary in rowsC:
        writer.writerow([dictionary['morph'], dictionary['pos'], dictionary['freq_all']])

In [21]:
df_statistica = pd.read_csv('statistica.csv')

df_statistica[:5]

,morph,POS,count
0,Case=Nom|Degree=Pos|Number=Plur,ADJ,49
1,Case=Nom|Degree=Pos|Gender=Masc|Number=Sing,ADJ,6
2,Case=Gen|Degree=Pos|Number=Plur,ADJ,40
3,Case=Gen|Degree=Pos|Gender=Masc|Number=Sing,ADJ,5
4,Case=Loc|Degree=Pos|Number=Plur,ADJ,10


In [22]:
len(df_statistica)

156637

In [23]:
df_statistica = df_statistica[df_statistica.morph != '_']

In [24]:
len(df_statistica)

151025

In [226]:
keys = zip(df_statistica['morph'], df_statistica['POS'], df_statistica['count'])
# freq = df_statistica['count']

In [227]:
dictn = {}
for key in keys:
    if (key[0], key[1]) in dictn:
        dictn[(key[0], key[1])] += key[2]
    else:
        dictn[(key[0], key[1])] = key[2]

In [228]:
len(dictn)

610

In [229]:
df_statistica = pd.Series(dictn).reset_index()
df_statistica.columns = ['tagset', 'POS', 'count'] 
df_statistica[:5]

,tagset,POS,count
0,Case=Nom|Degree=Pos|Number=Plur,ADJ,17467
1,Case=Nom|Degree=Pos|Gender=Masc|Number=Sing,ADJ,13906
2,Case=Gen|Degree=Pos|Number=Plur,ADJ,47265
3,Case=Gen|Degree=Pos|Gender=Masc|Number=Sing,ADJ,26364
4,Case=Loc|Degree=Pos|Number=Plur,ADJ,7511


In [230]:
df_statistica['part_freq'] = df_statistica['count'].apply(lambda x: x/df_statistica['count'].sum(axis=0))

In [231]:
df_statistica[:5]

,tagset,POS,count,part_freq
0,Case=Nom|Degree=Pos|Number=Plur,ADJ,17467,0.010666
1,Case=Nom|Degree=Pos|Gender=Masc|Number=Sing,ADJ,13906,0.008491
2,Case=Gen|Degree=Pos|Number=Plur,ADJ,47265,0.028861
3,Case=Gen|Degree=Pos|Gender=Masc|Number=Sing,ADJ,26364,0.016098
4,Case=Loc|Degree=Pos|Number=Plur,ADJ,7511,0.004586


In [232]:
df_statistica.to_csv(os.path.join(r"C:\Users\Andrea\desktop\part_freq.csv"))

In [233]:
if "morph_tags.csv" in os.getcwd():
    !del "morph_tags.csv" 

In [234]:
if "statistica.csv" in os.getcwd():
    !del "statistica.csv"

## Student texts
Now let's take some example student texts and extract tagsets and POS tagging using conllu API.

In [31]:
from conllu import parse, parse_tree

In [34]:
def parser(filename):
    """
    Yields a sentence from conllu tree with its tags

    """
    """
    >>> for i in parser('/content/gdrive/My Drive/Новые conll по доменам/NewVers/CleanedPsyEdu.conllu'):
      print(i)   
    TokenList<Музыка, звучит, отовсюду, независимо, от, нашего, желания, или, нежелания, слушать, ее, .>
    """
    with open(filename, 'r', encoding='utf-8') as f:
        data = f.read()
    tree = parse(data)
    for token in tree:
        yield token

In [36]:
def get_words(tree):
    """
    tree - generator of sentences (TokenLists) from conllu tree

    words, list is a list of all tokens we need from the tree
    size, int is a number of all words in the domain
    """
    words = []
    for sentence in tree:
        for token in sentence:
        # print(token)
            if token['form'] != '_' and token['upostag'] != '_' and token['upostag']!='NONLEX' and token['form'] not in r'[]\/':
                for wordform in token['form'].lower().split():
                    words.append((wordform, token['feats'], token['upostag']))
    size = len(words)
    return words, size

In [37]:
stud_dir = r'C:\Users\Andrea\Desktop\stud_textVSscie_text\Student_texts_for_experiments\stud_txt'
low_lvl = os.path.join(stud_dir, 'Low Level')
reg_lvl = os.path.join(stud_dir, 'Regular Level')
low_prsd = os.path.join(stud_dir, 'Low Level Parsed')
reg_prsd = os.path.join(stud_dir, 'Regular Level Parsed')

In [262]:
tree = parser(os.path.join(low_prsd, 'prs_EC12_B1_2421.conllu'))

In [263]:
words, size = get_words(tree)

In [264]:
del tree

In [265]:
size

455

In [266]:
words[:5]

[('<b1', None, 'NUM'),
 ('2421', None, 'NUM'),
 ('>', None, 'SYM'),
 ('<russian', OrderedDict([('Foreign', 'Yes')]), 'PROPN'),
 ('“н”=n', OrderedDict([('Foreign', 'Yes')]), 'PROPN')]

In [267]:
with open('tagset.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['token', 'tagset', 'POS'])

    for word in words:
        if word[1]:
            tag_lst = []
            for tag in list(word[1].items()):
                tag = '{}={}|'.format(tag[0], tag[1])
                tag_lst.append(tag)

            tag_str = ''.join([str(elem) for elem in tag_lst])
            tag_str = tag_str[:-1]

            writer.writerow([word[0], tag_str, word[2]])
        # print(tag_str)
        else:
            tag_str = 'None'
            writer.writerow([word[0], tag_str, word[2]]) 

In [268]:
df = pd.read_csv('tagset.csv')

In [269]:
df.drop(['token'], axis=1, inplace=True) 

In [270]:
dictn = {}

In [271]:
keys = zip(df['tagset'], df['POS'])
for couple in keys:
    if couple in dictn:
        dictn[couple] += 1
    else:
        dictn[couple] = 1

In [272]:
df_stat = pd.Series(dictn).reset_index()
df_stat.columns = ['tagset', 'POS', 'count'] 
df_stat[:5]

,tagset,POS,count
0,None,NUM,2
1,None,SYM,6
2,Foreign=Yes,PROPN,2
3,Degree=Pos,ADV,18
4,Case=Acc|Number=Plur|Person=2,PRON,1


In [273]:
df_stat['part_freq'] = df_stat['count'].apply(lambda x: x/df_stat['count'].sum(axis=0))

In [274]:
df_stat

,tagset,POS,count,part_freq
0,None,NUM,2,0.004396
1,None,SYM,6,0.013187
2,Foreign=Yes,PROPN,2,0.004396
3,Degree=Pos,ADV,18,0.039560
4,Case=Acc|Number=Plur|Person=2,PRON,1,0.002198
5,None,ADP,33,0.072527
6,Case=Gen|Number=Plur|Person=2,PRON,3,0.006593
7,None,PUNCT,79,0.173626
8,Case=Nom,PRON,4,0.008791
9,Aspect=Imp|Mood=Ind|Number=Sing|Person=3|Tense...,VERB,11,0.024176


In [160]:
df_statistica[:10]

,morph,POS,count,part_freq,x1000_freq
0,Case=Nom|Degree=Pos|Number=Plur,ADJ,49,0.000030,29.920119
1,Case=Nom|Degree=Pos|Gender=Masc|Number=Sing,ADJ,6,0.000004,3.663688
2,Case=Gen|Degree=Pos|Number=Plur,ADJ,40,0.000024,24.424587
3,Case=Gen|Degree=Pos|Gender=Masc|Number=Sing,ADJ,5,0.000003,3.053073
4,Case=Loc|Degree=Pos|Number=Plur,ADJ,10,0.000006,6.106147
5,Case=Acc|Degree=Pos|Gender=Fem|Number=Sing,ADJ,4,0.000002,2.442459
6,Case=Nom|Degree=Pos|Gender=Fem|Number=Sing,ADJ,5,0.000003,3.053073
7,Animacy=Inan|Case=Acc|Degree=Pos|Number=Plur,ADJ,7,0.000004,4.274303
8,Case=Gen|Degree=Pos|Gender=Fem|Number=Sing,ADJ,6,0.000004,3.663688
9,Case=Ins|Degree=Pos|Number=Plur,ADJ,4,0.000002,2.442459


In [275]:
df_stat['count'].sum(axis=0)

455

In [276]:
df_stat['x1000_freq'] = df_stat['count'].apply(lambda x: (x*1000)/df_stat['count'].sum(axis=0))

In [277]:
df_stat = df_stat.sort_values(by='POS').reset_index(drop=True)

In [278]:
df_stat[:10]

,tagset,POS,count,part_freq,x1000_freq
0,Case=Nom|Degree=Pos|Gender=Masc|Number=Sing,ADJ,12,0.026374,26.373626
1,Animacy=Inan|Case=Acc|Degree=Pos|Gender=Masc|N...,ADJ,4,0.008791,8.791209
2,Case=Nom|Degree=Pos|Gender=Neut|Number=Sing,ADJ,1,0.002198,2.197802
3,Degree=Pos|Gender=Fem|Number=Sing|Variant=Short,ADJ,1,0.002198,2.197802
4,Case=Acc|Degree=Pos|Gender=Neut|Number=Sing,ADJ,1,0.002198,2.197802
5,Degree=Pos|Gender=Masc|Number=Sing|Variant=Short,ADJ,1,0.002198,2.197802
6,Animacy=Inan|Case=Acc|Degree=Pos|Number=Plur,ADJ,1,0.002198,2.197802
7,Case=Gen|Degree=Pos|Number=Plur,ADJ,6,0.013187,13.186813
8,Case=Dat|Degree=Pos|Number=Plur,ADJ,1,0.002198,2.197802
9,Case=Loc|Degree=Pos|Number=Plur,ADJ,2,0.004396,4.395604


In [237]:
df_statistica['x1000_freq'] = df_statistica['count'].apply(lambda x: (x*1000)/df_statistica['count'].sum(axis=0))

In [238]:
df_statistica[:10]

,tagset,POS,count,part_freq,x1000_freq
0,Case=Nom|Degree=Pos|Number=Plur,ADJ,17467,0.010666,10.665607
1,Case=Nom|Degree=Pos|Gender=Masc|Number=Sing,ADJ,13906,0.008491,8.491208
2,Case=Gen|Degree=Pos|Number=Plur,ADJ,47265,0.028861,28.860703
3,Case=Gen|Degree=Pos|Gender=Masc|Number=Sing,ADJ,26364,0.016098,16.098245
4,Case=Loc|Degree=Pos|Number=Plur,ADJ,7511,0.004586,4.586327
5,Case=Acc|Degree=Pos|Gender=Fem|Number=Sing,ADJ,7714,0.004710,4.710282
6,Case=Nom|Degree=Pos|Gender=Fem|Number=Sing,ADJ,11949,0.007296,7.296235
7,Animacy=Inan|Case=Acc|Degree=Pos|Number=Plur,ADJ,10441,0.006375,6.375428
8,Case=Gen|Degree=Pos|Gender=Fem|Number=Sing,ADJ,33116,0.020221,20.221116
9,Case=Ins|Degree=Pos|Number=Plur,ADJ,8915,0.005444,5.443630


## Comparing statistics
Let's now try to compare the tagset frequency for our corpus and for the student text we selected.

In [285]:
corpus_kfreq = zip(df_statistica['tagset'], df_statistica['POS'], df_statistica['part_freq'])
corpus_dict = {(elem[0], elem[1]) : elem[2] for elem in corpus_kfreq}

In [286]:
stud_kfreq = zip(df_stat['tagset'], df_stat['POS'], df_stat['part_freq'])
stud_dict = {(elem[0], elem[1]) : elem[2] for elem in stud_kfreq}

In [287]:
diff_dict = {}
for elem in stud_dict:
    if elem in corpus_dict:
        diff = corpus_dict[elem] - stud_dict[elem]
    diff_dict[elem] = diff

In [288]:
len(diff_dict)

91

In [289]:
df_diff = pd.Series(diff_dict).reset_index()
df_diff.columns = ['tagset', 'POS', 'diff'] 
df_diff[:5]

,tagset,POS,diff
0,Case=Nom|Degree=Pos|Gender=Masc|Number=Sing,ADJ,-0.017882
1,Animacy=Inan|Case=Acc|Degree=Pos|Gender=Masc|N...,ADJ,-0.004503
2,Case=Nom|Degree=Pos|Gender=Neut|Number=Sing,ADJ,0.007819
3,Degree=Pos|Gender=Fem|Number=Sing|Variant=Short,ADJ,-0.001041
4,Case=Acc|Degree=Pos|Gender=Neut|Number=Sing,ADJ,0.002144


In [290]:
df_diff

,tagset,POS,diff
0,Case=Nom|Degree=Pos|Gender=Masc|Number=Sing,ADJ,-0.017882
1,Animacy=Inan|Case=Acc|Degree=Pos|Gender=Masc|N...,ADJ,-0.004503
2,Case=Nom|Degree=Pos|Gender=Neut|Number=Sing,ADJ,0.007819
3,Degree=Pos|Gender=Fem|Number=Sing|Variant=Short,ADJ,-0.001041
4,Case=Acc|Degree=Pos|Gender=Neut|Number=Sing,ADJ,0.002144
5,Degree=Pos|Gender=Masc|Number=Sing|Variant=Short,ADJ,-0.000745
6,Animacy=Inan|Case=Acc|Degree=Pos|Number=Plur,ADJ,0.004178
7,Case=Gen|Degree=Pos|Number=Plur,ADJ,0.015674
8,Case=Dat|Degree=Pos|Number=Plur,ADJ,0.000750
9,Case=Loc|Degree=Pos|Number=Plur,ADJ,0.000191


In [291]:
!del 'statistica.csv'

Impossibile trovare C:\Users\Andrea\CATandkittens_2019-2021\morphology\'statistica.csv'
